In [3]:
# On distingue deux types de caractéristiques : 
# - les caractéristiques continues dont toutes les valeurs sont ordonnées
# - les caractéristiques catégorielles dont les valeurs discrètes forment une énumération

In [6]:
# Pour faire tourner un algo d'apprentissage automatisé tel que la régression logistique, 
# on transforme l'énumération en autant de variables binaires ( dites indicatrices )
# mappant chaque terme à un chiffre binaire { rouge, vert, jaune } se transforme en {100,010,001}
# On appelle ça un encodage one-hot
from IPython.display import display
import os
import mglearn
import pandas as pd

adult_path = os.path.join(mglearn.datasets.DATA_PATH, "adult.data")
data = pd.read_csv( adult_path, header=None, index_col=False, names = [
    'age','workclass','fnlwgt','education','education-num', 'marital-status','occupation',
    'relationship','race','gender','capital-gain','capital-loss','hours-per-week',
    'native-coutry','income' ])
data = data[['age','workclass','education','gender','hours-per-week','occupation','income']]
display(data.head())

,age,workclass,education,gender,hours-per-week,occupation,income
0,39,State-gov,Bachelors,Male,40,Adm-clerical,<=50K
1,50,Self-emp-not-inc,Bachelors,Male,13,Exec-managerial,<=50K
2,38,Private,HS-grad,Male,40,Handlers-cleaners,<=50K
3,53,Private,11th,Male,40,Handlers-cleaners,<=50K
4,28,Private,Bachelors,Female,40,Prof-specialty,<=50K


In [11]:
# Les données énumérées doivent être standardisées, il y a autant de possiblités que d'orthographes possibles
# Les séries pandas peuvent aider à la vérification de la précision 
# https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html
print("Compte des valeurs prises par gender : \n")
print("{}".format(data.gender.value_counts()))

Compte des valeurs prises par gender : 

gender
 Male      21790
 Female    10771
Name: count, dtype: int64


In [17]:
# On peut encoder one-hot les données typées string grâce à get_dummies de pandas 
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html

print("Données originelles :")
print("{}\n".format(list(data.columns)))

data_dummies = pd.get_dummies(data)
print("Données compatibles :")
print("{}\n".format(list(data_dummies.columns)))

display(data_dummies.head(n=5))

Données originelles :
['age', 'workclass', 'education', 'gender', 'hours-per-week', 'occupation', 'income']

Données compatibles :
['age', 'hours-per-week', 'workclass_ ?', 'workclass_ Federal-gov', 'workclass_ Local-gov', 'workclass_ Never-worked', 'workclass_ Private', 'workclass_ Self-emp-inc', 'workclass_ Self-emp-not-inc', 'workclass_ State-gov', 'workclass_ Without-pay', 'education_ 10th', 'education_ 11th', 'education_ 12th', 'education_ 1st-4th', 'education_ 5th-6th', 'education_ 7th-8th', 'education_ 9th', 'education_ Assoc-acdm', 'education_ Assoc-voc', 'education_ Bachelors', 'education_ Doctorate', 'education_ HS-grad', 'education_ Masters', 'education_ Preschool', 'education_ Prof-school', 'education_ Some-college', 'gender_ Female', 'gender_ Male', 'occupation_ ?', 'occupation_ Adm-clerical', 'occupation_ Armed-Forces', 'occupation_ Craft-repair', 'occupation_ Exec-managerial', 'occupation_ Farming-fishing', 'occupation_ Handlers-cleaners', 'occupation_ Machine-op-inspct'

,age,hours-per-week,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,...,occupation_ Machine-op-inspct,occupation_ Other-service,occupation_ Priv-house-serv,occupation_ Prof-specialty,occupation_ Protective-serv,occupation_ Sales,occupation_ Tech-support,occupation_ Transport-moving,income_ <=50K,income_ >50K
0,39,40,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,True,False
1,50,13,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False
2,38,40,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False
3,53,40,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False
4,28,40,False,False,False,False,True,False,False,False,...,False,False,False,True,False,False,False,False,True,False


In [21]:
# On extrait les données de balisage 
caracteristiques = data_dummies.loc[:, 'age':'occupation_ Transport-moving']
X = caracteristiques.values
y = data_dummies['income_ >50K'].values
print("Shape X : {}, shape y : {}".format(X.shape,y.shape))

Shape X : (32561, 44), shape y : (32561,)


In [26]:
# On cherche a produire un modèle de régression logistique nous permettant de prédire si une personne
# aura un salaire au-dessus de 50K

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=0)
logreg = LogisticRegression(max_iter=10000)
logreg.fit(X_train,y_train)

print("Score sur les données de test  {}".format(logreg.score(X_test,y_test)))

Score sur les données de test  0.8093600294804079


In [28]:
# Pour le cas où les variables catégorielles sont déjà encodées avec des nombres ( entiers par exemple ),
# la fonction get_dummies traitera les variables comme continues.
# On peut
# - soit convertir les colonnes en chaine, 
# - soit utiliser OneHotEncoder de scikit-learn pour spécifier quelles colonnes considérer catégorielles

demo_df = pd.DataFrame({
    'Caractéristiques entières':[0,1,2,1],
    'Caractéristiques catégorielles': ['chaussette','poulet','chaussette','ballon']
})
display(demo_df)

,Caractéristiques entières,Caractéristiques catégorielles
0,0,chaussette
1,1,poulet
2,2,chaussette
3,1,ballon


In [30]:
# Les caractéristiques entières sont considérées continues
display(pd.get_dummies(demo_df))

,Caractéristiques entières,Caractéristiques catégorielles_ballon,Caractéristiques catégorielles_chaussette,Caractéristiques catégorielles_poulet
0,0,False,True,False
1,1,False,False,True
2,2,False,True,False
3,1,True,False,False


In [32]:
# Les caractéristiques entières sont considérées catégorielles
demo_df['Caractéristiques entières'] = demo_df['Caractéristiques entières'].astype(str)
display(pd.get_dummies(demo_df))

,Caractéristiques entières_0,Caractéristiques entières_1,Caractéristiques entières_2,Caractéristiques catégorielles_ballon,Caractéristiques catégorielles_chaussette,Caractéristiques catégorielles_poulet
0,True,False,False,False,True,False
1,False,True,False,False,False,True
2,False,False,True,False,True,False
3,False,True,False,True,False,False
